In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1.- Preprocesamiento de los datos

In [3]:
fdata = pd.read_csv('data_histories.csv', index_col="dp_folio")
fdata.head()

,id_sexo,Aedad,AAedad,Apuesto,id_gestud,AIMC,fuma,fuma_act,ejer_act,ejer1,...,locout5,locout10,locout20,locout30,rest_act,rest1,rest5,rest10,rest20,rest30
dp_folio,,,,,,,,,,,,,,,,,,,,,
1,F,51,6,Admin,CarTec,4,1,3,0,2,...,1,1,1,1,0,0,0,0,0,0
2,F,38,4,Sec,Bach,3,2,-1,0,0,...,0,0,0,0,1,3,0,0,0,0
3,F,34,3,Int,Sec,5,1,1,0,0,...,1,-1,-1,-1,0,0,0,-1,-1,-1
4,M,63,8,Jef,CarTec,4,2,-1,2,2,...,0,0,0,-1,0,0,0,0,0,0
5,M,42,4,EM,Sec,3,1,2,2,2,...,0,0,0,0,0,0,0,0,0,0


### Funciones para clusterizar/discretizar los datos

In [5]:
#Function to clusterize categories of a certain feature, and add the new clusterized feature as a new column
# Clusters should be an input of the form {cluster_A: {categories}, cluster_B: [categories]}
# Ex. obesity = {0:[1,2,3], 1:[4,5,6]}
def clusterizeDiscrete(feature, clusters, new_name, data):
    new_data = data.copy()
    original_list = data.loc[1:1080, feature]
    new_list = []
    for index in original_list.index:
        cat = False
        if original_list[index] == -1 or original_list[index] == "-1":
            new_list.append("N")
            continue
        for cluster in clusters:
            if original_list[index] in clusters[cluster]:
                new_list.append(cluster)
                cat = True
        if cat == False:
            new_list.append("N")
    new_data[new_name] = new_list
    return new_data

# Function to clusterize categories of a certain continous feature, and add the new clusterized feature as a
# new column
# Clusters shoud be an input of the form {cluster_A: {lambdaFunction1}, cluster_B: lambdaFunction2}
def clusterizeContinuous(feature, clusters, new_name, data):
    new_data = data.copy()
    original_list = data.loc[1:1080, feature]
    new_list = []
    for index in original_list.index:
        if original_list[index] == -1 or original_list[index] == "-1":
            new_list.append("N")
            continue
        for cluster in clusters:
            if eval(clusters[cluster])(original_list[index]):
                new_list.append(cluster)
                break
    new_data[new_name] = new_list
    return new_data
        

# Set of auxiliary high-order functions that will evaluate the conditions to binarize a history
def lessThan(num):
    return lambda n: n < num

def lessQThan(num):
    return lambda n: n <= num

def greaterThan(num):
    return lambda n: n > num

def greaterQThan(num):
    return lambda n: n >= num

def between(num1, num2):
    return lambda n: n <= num2 and n >= num1

### Definición y creación de los clusters

In [7]:
# Clusterize between obese and non obese
obesity = {0:[1,2,3], 1:[4,5,6]}
ndata = clusterizeDiscrete("AIMC", obesity, "obesity", fdata)

# Clusterize between degrees of study (higher degree vs. non higher degree)
degree = {0:["Prim", "Sec", "Bach", "CarTec", "Otro"], 1:["Lic", "Mast", "Doc", "PDoc"]}
ndata = clusterizeDiscrete("id_gestud", degree, "hdegree", ndata)

# Clusterize excercise features
ejer = {"A":"greaterQThan(2.5)", "B":"lessThan(2.5)"}
ndata = clusterizeContinuous("ejer_act", ejer, "ejer0B", ndata)
ndata = clusterizeContinuous("ejer1", ejer, "ejer1B", ndata)
ndata = clusterizeContinuous("ejer5", ejer, "ejer5B", ndata)
ndata = clusterizeContinuous("ejer10", ejer, "ejer10B", ndata)
ndata = clusterizeContinuous("ejer20", ejer, "ejer20B", ndata)
ndata = clusterizeContinuous("ejer30", ejer, "ejer30B", ndata)

# Clusterize stress features
estres = {"A":[4,5], "B":[1,2,3]}
ndata = clusterizeDiscrete("estres_act", estres, "estres0B", ndata)
ndata = clusterizeDiscrete("estres1", estres, "estres1B", ndata)
ndata = clusterizeDiscrete("estres5", estres, "estres5B", ndata)
ndata = clusterizeDiscrete("estres10", estres, "estres10B", ndata)
ndata = clusterizeDiscrete("estres20", estres, "estres20B", ndata)
ndata = clusterizeDiscrete("estres30", estres, "estres30B", ndata)

# Clusterize weight features
peso = {"A":[1,2,3], "B":[4,5]}
ndata = clusterizeDiscrete("peso_act", peso, "peso0B", ndata)
ndata = clusterizeDiscrete("peso1", peso, "peso1B", ndata)
ndata = clusterizeDiscrete("peso5", peso, "peso5B", ndata)
ndata = clusterizeDiscrete("peso10", peso, "peso10B", ndata)
ndata = clusterizeDiscrete("peso20", peso, "peso20B", ndata)
ndata = clusterizeDiscrete("peso30", peso, "peso30B", ndata)

# Clusterize weight features
condi = {"A":[4,5], "B":[1,2,3]}
ndata = clusterizeDiscrete("condi_act", condi, "condi0B", ndata)
ndata = clusterizeDiscrete("condi1", condi, "condi1B", ndata)
ndata = clusterizeDiscrete("condi5", condi, "condi5B", ndata)
ndata = clusterizeDiscrete("condi10", condi, "condi10B", ndata)
ndata = clusterizeDiscrete("condi20", condi, "condi20B", ndata)
ndata = clusterizeDiscrete("condi30", condi, "condi30B", ndata)

# Clusterize health features
salud = {"A":[4,5], "B":[1,2,3]}
ndata = clusterizeDiscrete("salud_act", salud, "salud0B", ndata)
ndata = clusterizeDiscrete("salud1", salud, "salud1B", ndata)
ndata = clusterizeDiscrete("salud5", salud, "salud5B", ndata)
ndata = clusterizeDiscrete("salud10", salud, "salud10B", ndata)
ndata = clusterizeDiscrete("salud20", salud, "salud20B", ndata)
ndata = clusterizeDiscrete("salud30", salud, "salud30B", ndata)

# Clusterize job features
academic = {0:["Admin", "Asi", "Coo", "E", "ED", "EM", "Int", "Jef", "Lab", "Sec", "Tec", "Vig"], 1:["Acade", "Inv", "InvE"]}
ndata = clusterizeDiscrete("Apuesto", academic, "academic", ndata)

# Clusterize walking features
walking = {"A":"greaterQThan(1800.0)", "B":"lessThan(1800.0)"}
ndata = clusterizeContinuous("dis_dia", walking, "dis_dia0B", ndata)
ndata = clusterizeContinuous("dis1_dia", walking, "dis_dia1B", ndata)
ndata = clusterizeContinuous("dis5_dia", walking, "dis_dia5B", ndata)
ndata = clusterizeContinuous("dis10_dia", walking, "dis_dia10B", ndata)
ndata = clusterizeContinuous("dis20_dia", walking, "dis_dia20B", ndata)
ndata = clusterizeContinuous("dis30_dia", walking, "dis_dia30B", ndata)


# Clusterize age features
age = {0:"between(15.0, 28.0)", 1:"between(28.1, 40.0)", 2:"between(40.1, 60.0)", 3:"between(60.1, 90.0)"}
ndata = clusterizeContinuous("Aedad", age, "AedadC", ndata)

In [8]:
ndata.head()

,id_sexo,Aedad,AAedad,Apuesto,id_gestud,AIMC,fuma,fuma_act,ejer_act,ejer1,...,salud20B,salud30B,academic,dis_dia0B,dis_dia1B,dis_dia5B,dis_dia10B,dis_dia20B,dis_dia30B,AedadC
dp_folio,,,,,,,,,,,,,,,,,,,,,
1,F,51,6,Admin,CarTec,4,1,3,0,2,...,A,A,0,B,B,A,N,N,N,2
2,F,38,4,Sec,Bach,3,2,-1,0,0,...,A,A,0,A,B,B,B,A,B,1
3,F,34,3,Int,Sec,5,1,1,0,0,...,A,N,0,B,B,B,B,B,B,1
4,M,63,8,Jef,CarTec,4,2,-1,2,2,...,A,A,0,A,A,A,A,A,A,3
5,M,42,4,EM,Sec,3,1,2,2,2,...,N,N,0,B,B,B,N,N,N,2


### Creación de la matriz de variables (features)

In [53]:
# Build profile based on excercise, health and stress in the last 0, 1, 5 and 10 years
profiles_0_to_10 = ndata[["ejer0B", "salud0B", "estres0B", "ejer1B", "salud1B", "estres1B", "ejer5B", "salud5B", "estres5B", "ejer10B", "salud10B", "estres10B"]]
profiles_1_to_10 = ndata[["ejer1B", "salud1B", "estres1B", "ejer1B", "salud1B", "estres1B", "ejer5B", "salud5B", "estres5B", "ejer10B", "salud10B", "estres10B"]]

# Replace "A" with 1, and "B" with 0, in order to have binary values, and save this in the feature matrix X
X_0 = profiles_0_to_10.replace("A", 1).replace("B", 0).replace("N", 0)
X_1 = profiles_1_to_10.replace("A", 1).replace("B", 0).replace("N", 0)

In [54]:
X_0.head()

,ejer0B,salud0B,estres0B,ejer1B,salud1B,estres1B,ejer5B,salud5B,estres5B,ejer10B,salud10B,estres10B
dp_folio,,,,,,,,,,,,
1,0,1,1,0,1,0,0,1,1,0,1,0
2,0,1,0,0,1,0,1,0,1,1,1,0
3,0,0,1,0,0,1,0,1,1,1,1,1
4,0,1,0,0,1,1,1,1,0,1,1,0
5,0,1,0,0,1,0,0,0,0,0,0,1


In [55]:
X_1.head()

,ejer1B,salud1B,estres1B,ejer1B,salud1B,estres1B,ejer5B,salud5B,estres5B,ejer10B,salud10B,estres10B
dp_folio,,,,,,,,,,,,
1,0,1,0,0,1,0,0,1,1,0,1,0
2,0,1,0,0,1,0,1,0,1,1,1,0
3,0,0,1,0,0,1,0,1,1,1,1,1
4,0,1,1,0,1,1,1,1,0,1,1,0
5,0,1,0,0,1,0,0,0,0,0,0,1


## 2.- RBM

In [164]:
from sklearn.neural_network import BernoulliRBM

In [427]:
rbm = BernoulliRBM(verbose=True, batch_size = 10, learning_rate=0.001, n_iter=1000, n_components = 100)
rbm.fit(X_0)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -7.81, time = 0.02s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -7.67, time = 0.03s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -7.69, time = 0.02s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -7.43, time = 0.02s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -7.52, time = 0.02s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -7.52, time = 0.02s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -7.69, time = 0.02s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -7.58, time = 0.02s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -7.72, time = 0.02s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -7.77, time = 0.02s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -7.60, time = 0.04s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -7.48, time = 0.02s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -7.73, time = 0.02s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -7.55, time = 0.02s
[BernoulliRBM] Iteration 15, pseudo-likelih

[BernoulliRBM] Iteration 124, pseudo-likelihood = -7.63, time = 0.03s
[BernoulliRBM] Iteration 125, pseudo-likelihood = -7.54, time = 0.03s
[BernoulliRBM] Iteration 126, pseudo-likelihood = -7.48, time = 0.02s
[BernoulliRBM] Iteration 127, pseudo-likelihood = -7.72, time = 0.02s
[BernoulliRBM] Iteration 128, pseudo-likelihood = -7.42, time = 0.02s
[BernoulliRBM] Iteration 129, pseudo-likelihood = -7.52, time = 0.02s
[BernoulliRBM] Iteration 130, pseudo-likelihood = -7.66, time = 0.02s
[BernoulliRBM] Iteration 131, pseudo-likelihood = -7.42, time = 0.02s
[BernoulliRBM] Iteration 132, pseudo-likelihood = -7.67, time = 0.02s
[BernoulliRBM] Iteration 133, pseudo-likelihood = -7.36, time = 0.04s
[BernoulliRBM] Iteration 134, pseudo-likelihood = -7.72, time = 0.06s
[BernoulliRBM] Iteration 135, pseudo-likelihood = -7.47, time = 0.07s
[BernoulliRBM] Iteration 136, pseudo-likelihood = -7.58, time = 0.06s
[BernoulliRBM] Iteration 137, pseudo-likelihood = -7.54, time = 0.06s
[BernoulliRBM] Itera

[BernoulliRBM] Iteration 249, pseudo-likelihood = -7.63, time = 0.02s
[BernoulliRBM] Iteration 250, pseudo-likelihood = -7.33, time = 0.03s
[BernoulliRBM] Iteration 251, pseudo-likelihood = -7.46, time = 0.03s
[BernoulliRBM] Iteration 252, pseudo-likelihood = -7.43, time = 0.02s
[BernoulliRBM] Iteration 253, pseudo-likelihood = -7.29, time = 0.02s
[BernoulliRBM] Iteration 254, pseudo-likelihood = -7.65, time = 0.03s
[BernoulliRBM] Iteration 255, pseudo-likelihood = -7.45, time = 0.03s
[BernoulliRBM] Iteration 256, pseudo-likelihood = -7.44, time = 0.02s
[BernoulliRBM] Iteration 257, pseudo-likelihood = -7.48, time = 0.02s
[BernoulliRBM] Iteration 258, pseudo-likelihood = -7.52, time = 0.03s
[BernoulliRBM] Iteration 259, pseudo-likelihood = -7.42, time = 0.03s
[BernoulliRBM] Iteration 260, pseudo-likelihood = -7.26, time = 0.02s
[BernoulliRBM] Iteration 261, pseudo-likelihood = -7.44, time = 0.02s
[BernoulliRBM] Iteration 262, pseudo-likelihood = -7.43, time = 0.02s
[BernoulliRBM] Itera

[BernoulliRBM] Iteration 374, pseudo-likelihood = -7.18, time = 0.07s
[BernoulliRBM] Iteration 375, pseudo-likelihood = -6.92, time = 0.04s
[BernoulliRBM] Iteration 376, pseudo-likelihood = -7.21, time = 0.03s
[BernoulliRBM] Iteration 377, pseudo-likelihood = -7.14, time = 0.02s
[BernoulliRBM] Iteration 378, pseudo-likelihood = -7.32, time = 0.02s
[BernoulliRBM] Iteration 379, pseudo-likelihood = -7.14, time = 0.02s
[BernoulliRBM] Iteration 380, pseudo-likelihood = -7.31, time = 0.02s
[BernoulliRBM] Iteration 381, pseudo-likelihood = -6.85, time = 0.02s
[BernoulliRBM] Iteration 382, pseudo-likelihood = -7.13, time = 0.02s
[BernoulliRBM] Iteration 383, pseudo-likelihood = -7.01, time = 0.03s
[BernoulliRBM] Iteration 384, pseudo-likelihood = -6.99, time = 0.03s
[BernoulliRBM] Iteration 385, pseudo-likelihood = -7.00, time = 0.03s
[BernoulliRBM] Iteration 386, pseudo-likelihood = -7.21, time = 0.02s
[BernoulliRBM] Iteration 387, pseudo-likelihood = -7.14, time = 0.02s
[BernoulliRBM] Itera

[BernoulliRBM] Iteration 496, pseudo-likelihood = -6.89, time = 0.03s
[BernoulliRBM] Iteration 497, pseudo-likelihood = -6.75, time = 0.04s
[BernoulliRBM] Iteration 498, pseudo-likelihood = -6.74, time = 0.02s
[BernoulliRBM] Iteration 499, pseudo-likelihood = -6.88, time = 0.03s
[BernoulliRBM] Iteration 500, pseudo-likelihood = -6.82, time = 0.02s
[BernoulliRBM] Iteration 501, pseudo-likelihood = -6.86, time = 0.02s
[BernoulliRBM] Iteration 502, pseudo-likelihood = -7.16, time = 0.03s
[BernoulliRBM] Iteration 503, pseudo-likelihood = -6.76, time = 0.02s
[BernoulliRBM] Iteration 504, pseudo-likelihood = -6.59, time = 0.03s
[BernoulliRBM] Iteration 505, pseudo-likelihood = -6.84, time = 0.03s
[BernoulliRBM] Iteration 506, pseudo-likelihood = -6.79, time = 0.04s
[BernoulliRBM] Iteration 507, pseudo-likelihood = -6.81, time = 0.03s
[BernoulliRBM] Iteration 508, pseudo-likelihood = -6.87, time = 0.03s
[BernoulliRBM] Iteration 509, pseudo-likelihood = -6.57, time = 0.03s
[BernoulliRBM] Itera

[BernoulliRBM] Iteration 622, pseudo-likelihood = -6.46, time = 0.02s
[BernoulliRBM] Iteration 623, pseudo-likelihood = -6.19, time = 0.04s
[BernoulliRBM] Iteration 624, pseudo-likelihood = -6.51, time = 0.02s
[BernoulliRBM] Iteration 625, pseudo-likelihood = -6.70, time = 0.02s
[BernoulliRBM] Iteration 626, pseudo-likelihood = -6.59, time = 0.02s
[BernoulliRBM] Iteration 627, pseudo-likelihood = -6.45, time = 0.02s
[BernoulliRBM] Iteration 628, pseudo-likelihood = -6.49, time = 0.02s
[BernoulliRBM] Iteration 629, pseudo-likelihood = -6.41, time = 0.02s
[BernoulliRBM] Iteration 630, pseudo-likelihood = -6.84, time = 0.02s
[BernoulliRBM] Iteration 631, pseudo-likelihood = -6.35, time = 0.02s
[BernoulliRBM] Iteration 632, pseudo-likelihood = -6.69, time = 0.03s
[BernoulliRBM] Iteration 633, pseudo-likelihood = -6.60, time = 0.03s
[BernoulliRBM] Iteration 634, pseudo-likelihood = -6.43, time = 0.02s
[BernoulliRBM] Iteration 635, pseudo-likelihood = -6.45, time = 0.02s
[BernoulliRBM] Itera

[BernoulliRBM] Iteration 740, pseudo-likelihood = -6.41, time = 0.03s
[BernoulliRBM] Iteration 741, pseudo-likelihood = -6.72, time = 0.02s
[BernoulliRBM] Iteration 742, pseudo-likelihood = -6.58, time = 0.03s
[BernoulliRBM] Iteration 743, pseudo-likelihood = -6.36, time = 0.02s
[BernoulliRBM] Iteration 744, pseudo-likelihood = -6.34, time = 0.02s
[BernoulliRBM] Iteration 745, pseudo-likelihood = -6.59, time = 0.02s
[BernoulliRBM] Iteration 746, pseudo-likelihood = -6.43, time = 0.02s
[BernoulliRBM] Iteration 747, pseudo-likelihood = -6.36, time = 0.02s
[BernoulliRBM] Iteration 748, pseudo-likelihood = -6.40, time = 0.02s
[BernoulliRBM] Iteration 749, pseudo-likelihood = -6.27, time = 0.03s
[BernoulliRBM] Iteration 750, pseudo-likelihood = -6.42, time = 0.03s
[BernoulliRBM] Iteration 751, pseudo-likelihood = -6.31, time = 0.03s
[BernoulliRBM] Iteration 752, pseudo-likelihood = -6.41, time = 0.02s
[BernoulliRBM] Iteration 753, pseudo-likelihood = -6.40, time = 0.02s
[BernoulliRBM] Itera

[BernoulliRBM] Iteration 859, pseudo-likelihood = -6.33, time = 0.04s
[BernoulliRBM] Iteration 860, pseudo-likelihood = -6.07, time = 0.02s
[BernoulliRBM] Iteration 861, pseudo-likelihood = -6.12, time = 0.02s
[BernoulliRBM] Iteration 862, pseudo-likelihood = -6.31, time = 0.02s
[BernoulliRBM] Iteration 863, pseudo-likelihood = -6.14, time = 0.02s
[BernoulliRBM] Iteration 864, pseudo-likelihood = -6.37, time = 0.02s
[BernoulliRBM] Iteration 865, pseudo-likelihood = -5.86, time = 0.02s
[BernoulliRBM] Iteration 866, pseudo-likelihood = -6.26, time = 0.02s
[BernoulliRBM] Iteration 867, pseudo-likelihood = -6.06, time = 0.02s
[BernoulliRBM] Iteration 868, pseudo-likelihood = -6.18, time = 0.02s
[BernoulliRBM] Iteration 869, pseudo-likelihood = -6.24, time = 0.04s
[BernoulliRBM] Iteration 870, pseudo-likelihood = -6.25, time = 0.03s
[BernoulliRBM] Iteration 871, pseudo-likelihood = -6.15, time = 0.02s
[BernoulliRBM] Iteration 872, pseudo-likelihood = -6.07, time = 0.02s
[BernoulliRBM] Itera

[BernoulliRBM] Iteration 978, pseudo-likelihood = -6.07, time = 0.02s
[BernoulliRBM] Iteration 979, pseudo-likelihood = -6.00, time = 0.04s
[BernoulliRBM] Iteration 980, pseudo-likelihood = -5.71, time = 0.03s
[BernoulliRBM] Iteration 981, pseudo-likelihood = -5.95, time = 0.03s
[BernoulliRBM] Iteration 982, pseudo-likelihood = -5.90, time = 0.03s
[BernoulliRBM] Iteration 983, pseudo-likelihood = -6.11, time = 0.03s
[BernoulliRBM] Iteration 984, pseudo-likelihood = -5.83, time = 0.03s
[BernoulliRBM] Iteration 985, pseudo-likelihood = -5.96, time = 0.02s
[BernoulliRBM] Iteration 986, pseudo-likelihood = -6.02, time = 0.05s
[BernoulliRBM] Iteration 987, pseudo-likelihood = -6.13, time = 0.03s
[BernoulliRBM] Iteration 988, pseudo-likelihood = -6.22, time = 0.02s
[BernoulliRBM] Iteration 989, pseudo-likelihood = -6.06, time = 0.02s
[BernoulliRBM] Iteration 990, pseudo-likelihood = -5.97, time = 0.02s
[BernoulliRBM] Iteration 991, pseudo-likelihood = -6.08, time = 0.02s
[BernoulliRBM] Itera

BernoulliRBM(batch_size=10, learning_rate=0.001, n_components=100,
       n_iter=1000, random_state=None, verbose=True)

In [236]:
rbm.transform(X_1)[0]

array([0.60388188, 0.63812917, 0.63183367, 0.62320239, 0.62269582,
       0.63145465, 0.62940669, 0.61893685])

## 2.- RBM

In [324]:
def batch_iterator(X, y=None, batch_size=64):
    n_samples = X.shape[0]
    for i in np.arange(0, n_samples, batch_size):
        begin, end = i, min(i+batch_size, n_samples)
        if y is not None:
            yield X[begin:end], y[begin:end]
        else:
            yield X[begin:end]

In [561]:
class RBM :
    
    def __init__(self, num_visible, num_hidden, learning_rate, batch_size, num_epochs):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        
        self.W = np.random.normal(scale=0.1, size=(num_visible, num_hidden))
        self.a = np.zeros(num_visible)
        self.b = np.zeros(num_hidden)
    
    # Calculate the sigmoid of X 
    def sigmoid(self, X):
        return 1 / (1 + np.exp(-1*X))
    
    # Sample the activations given a certain matrix of probabilities
    def sample(self, X):
        return X > np.random.random_sample(size=X.shape)
    
    # Perform a reconstruction of the input data X
    def gibbs_sample(self, X):
        # Positive phase: Calculate the activations of the hidden layer
        positive_hidden = self.sigmoid(X.dot(self.W) + self.b)
        hidden_states = self.sample(positive_hidden)
        # Negative phase: Given the activations of the hidden layer, reconstruct the states at the visible layer
        negative_visible = self.sigmoid(hidden_states.dot(self.W.T) + self.a)
        visible_states = self.sample(negative_visible)
        return visible_states
    
    # Get the hidden probabilities for a certain input
    def transform(self, X):
        return self.sigmoid(X.dot(self.W) + self.b)
    
    def train(self, X):
        
        # Define matrix to keep track of the training MSE
        self.training_errors = []
        
        for epoch in range(self.num_epochs):
            
            batch_errors = []
            
            for batch in batch_iterator(X, batch_size=self.batch_size):
                # Positive phase: Calculate the activations of the hidden layer
                positive_hidden_probs = self.sigmoid(batch.dot(self.W) + self.b)
                positive_hidden_states = self.sample(positive_hidden_probs)
                
                # Calculate vh_data using the positive hidden states activations, rather than their probabilities
                # as per Hinton (2010)
                vh_data = batch.T.dot(positive_hidden_probs)
                
                # Negative phase
                negative_visible_probs = self.sigmoid(positive_hidden_states.dot(self.W.T) + self.a)
                negative_visible_states = self.sample(negative_visible_probs)
                negative_hidden_probs = self.sigmoid(negative_visible_states.dot(self.W) + self.b)
                negative_hidden_states = self.sample(negative_hidden_probs)
                
                # Calculate vh_reconstruction using the negative hidden states activations
                vh_reconstruction = negative_visible_states.T.dot(negative_hidden_probs)
                
                # Update weights and biases
                self.W += self.learning_rate * (vh_data - vh_reconstruction)
                self.b += self.learning_rate * (positive_hidden_probs.sum(axis=0) - negative_hidden_probs.sum(axis=0))
                self.a += self.learning_rate * (batch.sum(axis=0) - negative_visible_probs.sum(axis=0))
                
                batch_errors.append(np.mean((batch - negative_visible_states) ** 2))
                                    
            self.training_errors.append(np.mean(batch_errors))
                

In [562]:
rbm = RBM(12, 100, 0.001, 10, 1000)

In [563]:
rbm.train(X_0.values)

In [605]:
#rbm.training_errors

In [504]:
print X_0.iloc[0].values
print rbm.transform(X_0.iloc[0])
print rbm.gibbs_sample(X_0.iloc[0])

[0 1 1 0 1 0 0 1 1 0 1 0]
[0.04437584 0.54609641 0.62182498 0.88072227 0.35071295 0.2307675
 0.10376816 0.39774787 0.14185395 0.06489129 0.54243639 0.32237775
 0.75760049 0.11865689 0.41162725 0.39600649 0.71697328 0.06747624
 0.21773576 0.7412642  0.32221094 0.37515107 0.70299606 0.38636326
 0.55348903 0.31454731 0.60153548 0.18201609 0.41920545 0.7881072
 0.14344384 0.41413304 0.14633159 0.18712117 0.18115877 0.11206997
 0.64559909 0.28885965 0.24145856 0.19260524 0.5013285  0.6326868
 0.66690297 0.31079777 0.46934398 0.11360373 0.40616838 0.36347016
 0.44510102 0.47243354 0.53099696 0.7486797  0.53073609 0.37642273
 0.11161793 0.49869613 0.21931415 0.59677996 0.02150337 0.02652341
 0.25981895 0.71208058 0.53077567 0.4164095  0.2438594  0.69221503
 0.17826946 0.08481603 0.27928511 0.66727936 0.32211962 0.14616702
 0.61352713 0.43420388 0.35653394 0.06461007 0.09371335 0.22469556
 0.7292419  0.74987574 0.27091999 0.20423742 0.22436842 0.26884069
 0.74180191 0.5456513  0.56346531 0.410

## 3.- CRBM

In [624]:
class CRBM :
    
    def __init__(self, num_visible, num_hidden, num_historic, learning_rate, batch_size, num_epochs):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.num_historic = num_historic
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        
        # Initialize weights and biases
        self.W = np.random.normal(scale=0.1, size=(num_visible, num_hidden))
        self.a = np.zeros(num_visible)
        self.b = np.zeros(num_hidden)
        
        # Initialize autoregreesive parameters
        self.A = np.random.normal(scale=0.01, size=(num_visible, num_historic))
        self.B = np.random.normal(scale=0.01, size=(num_hidden, num_historic))
    
    # Calculate the sigmoid of X 
    def sigmoid(self, X):
        return 1 / (1 + np.exp(-1*X))
    
    # Sample the activations given a certain matrix of probabilities
    def sample(self, X):
        return X > np.random.random_sample(size=X.shape)
    
    # Perform a reconstruction of the input data X
    def gibbs_sample(self, X):
        
        # Separate input vector
        V = np.array(X.values[:, :3])
        H = np.array(X.values[:, 3:])

        # Positive phase

        # Calculate the contributions from the Historic layer
        dinamic_b = self.b + H.dot(self.B.T)

        # Calculate the activations of the Hidden layer
        positive_hidden = sigmoid(V.dot(self.W) + dinamic_b)
        hidden_states = sample(positive_hidden)

        # Negative phase

        #Calculate the contributions from the Historic layer
        dinamic_a = self.a + H.dot(self.A.T)
        
        # Calculate the activations of the Visible layer
        negative_visible = sigmoid(hidden_states.dot(self.W.T) + dinamic_a)
        visible_states = sample(negative_visible)
        return visible_states
    
    # Get the hidden probabilities for a certain input
    def transform(self, X):
        return self.sigmoid(X.dot(self.W) + self.b)
    
    def train(self, X):
        
        # Define matrix to keep track of the training MSE
        self.training_errors = []
        
        for epoch in range(self.num_epochs):
            
            batch_errors = []
            
            for batch in batch_iterator(X, batch_size=self.batch_size):
                
                # Separate input vector
                V = np.array(batch.values[:, :3])
                H = np.array(batch.values[:, 3:])
                
                
                # Positive phase
                # Calculate the contributions from the Historic layer
                dinamic_b = self.b + H.dot(self.B.T)
                
                # Calculate the activations of the hidden layer
                positive_hidden_probs = self.sigmoid(V.dot(self.W) + dinamic_b)
                positive_hidden_states = self.sample(positive_hidden_probs)
                
                # Calculate vh_data using the positive hidden probabilities, rather than their activations
                # as per Hinton (2010)
                vh_data = V.T.dot(positive_hidden_probs)
                
                # Calculate vH_data (H: Historic layer)
                vH_data = V.T.dot(H)
                # Calculate hH_data (H: Historic layer)
                hH_data = positive_hidden_states.T.dot(H)
                
                
                
                # Negative phase
                
                # Calculate the contributions from the Historic layer
                dinamic_a = self.a + H.dot(self.A.T)
                
                negative_visible_probs = self.sigmoid(positive_hidden_states.dot(self.W.T) + dinamic_a)
                negative_visible_states = self.sample(negative_visible_probs)
                negative_hidden_probs = self.sigmoid(negative_visible_states.dot(self.W) + dinamic_b)
                negative_hidden_states = self.sample(negative_hidden_probs)
                
                # Calculate vh_reconstruction using the negative hidden states probabilities
                vh_reconstruction = negative_visible_states.T.dot(negative_hidden_probs)
                
                # Calculate vH_reconstruction (H: Historic layer)
                vH_reconstruction = negative_visible_states.T.dot(H)
                # Calculate hH_reconstruction (H: Historic layer)
                hH_reconstruction = negative_hidden_states.T.dot(H)
                
                # Update weights and biases
                self.W += self.learning_rate * (vh_data - vh_reconstruction)
                self.b += self.learning_rate * (positive_hidden_probs.sum(axis=0) - negative_hidden_probs.sum(axis=0))
                self.a += self.learning_rate * (V.sum(axis=0) - negative_visible_probs.sum(axis=0))
                self.B += self.learning_rate * 0.01 * (hH_data - hH_reconstruction)
                self.A += self.learning_rate * 0.01 * (vH_data - vH_reconstruction)
                
                batch_errors.append(np.mean((V - negative_visible_states) ** 2))
                                    
            self.training_errors.append(np.mean(batch_errors))
                

In [625]:
crbm = CRBM(3, 100, 9, 0.001, 10, 1000)

In [626]:
crbm.train(X_0)

In [627]:
crbm.training_errors

[0.4689300411522634,
 0.462551440329218,
 0.4663580246913581,
 0.4531893004115226,
 0.47160493827160493,
 0.45637860082304527,
 0.4593621399176955,
 0.44619341563786,
 0.4647119341563786,
 0.4657407407407408,
 0.4527777777777778,
 0.4574074074074075,
 0.44382716049382714,
 0.45596707818930043,
 0.4632716049382716,
 0.4525720164609053,
 0.44012345679012355,
 0.4515432098765431,
 0.46213991769547325,
 0.45267489711934156,
 0.4588477366255143,
 0.45401234567901233,
 0.45082304526748973,
 0.4593621399176955,
 0.4435185185185185,
 0.44053497942386827,
 0.4266460905349795,
 0.46069958847736625,
 0.4441358024691358,
 0.4437242798353909,
 0.44989711934156373,
 0.4489711934156378,
 0.45133744855967073,
 0.42901234567901236,
 0.4543209876543211,
 0.4543209876543211,
 0.45493827160493827,
 0.4403292181069959,
 0.4445473251028806,
 0.43641975308641984,
 0.4280864197530864,
 0.43065843621399175,
 0.450514403292181,
 0.4369341563786008,
 0.44166666666666665,
 0.42026748971193406,
 0.4329218106995885

## 4.- SRBM

### Tests

In [246]:
np_rng = np.random.RandomState()
weights = 0.1 * np.random.rand(3, 2)
weights

array([[0.03158819, 0.01143351],
       [0.00914015, 0.06884196],
       [0.09950659, 0.04133751]])

In [295]:
y = np.array([0.2, 0.8])
y > np.random.random_sample(size=y.shape)

array([ True,  True])

In [312]:
rbm = RBM(12, 10, 0.1, 10, 1000)

In [326]:
rbm.gibbs_sample(X_0.iloc[0].values)

array([False,  True,  True, False,  True,  True, False,  True, False,
       False,  True, False])

In [602]:
#for batch in batch_iterator(X_0, batch_size=100):
#    print batch

In [512]:
X_0.values.sum(axis=0)

array([424, 631, 506, 465, 647, 552, 565, 813, 360, 593, 903, 207])

In [608]:
print X_0.values[:, :3].shape
print X_0.values[:, 3:].shape

(1076, 3)
(1076, 9)


In [612]:
#X_0.iloc[:, 3:]

In [554]:
num_visible = 3
num_hidden = 10
num_historic = 9

# Initialize weights and biases
W = np.random.normal(scale=0.1, size=(num_visible, num_hidden))
a = np.zeros(num_visible)
b = np.zeros(num_hidden)
        
# Initialize autoregreesive parameters
A = np.random.normal(scale=0.01, size=(num_visible, num_historic))
B = np.random.normal(scale=0.01, size=(num_hidden, num_historic))

# Calculate the sigmoid of X 
def sigmoid(X):
    return 1 / (1 + np.exp(-1*X))
    
# Sample the activations given a certain matrix of probabilities
def sample(X):
    return X > np.random.random_sample(size=X.shape)

# Perform a reconstruction of the input data X
def gibbs_sample(X):
        
    # Separate input vector
    V = np.array(X.values[:, :3])
    H = np.array(X.values[:, 3:])
        
    # Positive phase
        
    # Calculate the contributions from the Historic layer
    dinamic_b = b + H.dot(B.T).sum(axis=0)
    print b
    print dinamic_b
        
    positive_hidden = sigmoid(V.dot(W) + dinamic_b)
    hidden_states = sample(positive_hidden)
        
    # Negative phase
        
    #Calculate the contributions from the Historic layer
    dinamic_a = a + H.dot(A.T).sum(axis=0)
        
    negative_visible = sigmoid(hidden_states.dot(W.T) + dinamic_a)
    visible_states = sample(negative_visible)
    return visible_states

In [603]:
#gibbs_sample(X_0)